In [1]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
text = "2023 will be a great year for all of us"

encoding = tokenizer.encode_plus(text,add_special_tokens=True,truncation=True,padding='max_length',return_attention_mask=True,return_tensors='pt')

In [6]:
encoding

{'input_ids': tensor([[  101, 16798,  2509,  2097,  2022,  1037,  2307,  2095,  2005,  2035,
          1997,  2149,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [7]:
encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [8]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [9]:
tokenizer.mask_token

'[MASK]'

In [10]:
tokenizer.mask_token_id

103

In [14]:
tokenizer.vocab['a']

1037

In [15]:
from transformers import BertForMaskedLM
from torch.nn import functional as F
import torch

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased',return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
model #LM head applied on top of BERT architecture.

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [72]:
text = "The opera house in Australia is in, " + tokenizer.mask_token + ' city'

input = tokenizer.encode_plus(text,return_tensors='pt')

In [73]:
input

{'input_ids': tensor([[ 101, 1996, 3850, 2160, 1999, 2660, 2003, 1999, 1010,  103, 2103,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [74]:
mask_index = torch.where(input['input_ids'][0] == tokenizer.mask_token_id)

In [75]:
output = model(**input)

In [76]:
output

MaskedLMOutput(loss=None, logits=tensor([[[ -6.4283,  -6.4008,  -6.4152,  ...,  -5.8517,  -5.6598,  -3.8024],
         [-14.3433, -14.1573, -14.4389,  ..., -12.4283, -12.1888, -14.0083],
         [ -8.1241,  -8.1406,  -8.0270,  ...,  -6.3535,  -8.0919,  -8.6380],
         ...,
         [ -5.5191,  -5.4260,  -5.2918,  ...,  -5.7274,  -6.7980,  -3.8485],
         [ -9.4287,  -9.1882,  -9.3683,  ...,  -8.0449,  -8.3085,  -6.5527],
         [-10.8957, -10.4429, -10.6936,  ...,  -8.8795,  -9.7457,  -7.8003]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [77]:
output.logits.shape #(Batch_size, tokenized_text_length, vocab_size)

torch.Size([1, 12, 30522])

In [78]:
logits = output.logits

In [79]:
softmax = F.softmax(logits,dim=-1)
mask_word = softmax[0, mask_index, :]

In [80]:
mask_word

tensor([[6.5519e-08, 7.1913e-08, 8.2243e-08,  ..., 5.3200e-08, 1.8237e-08,
         3.4827e-07]], grad_fn=<IndexBackward0>)

In [85]:
top_10 = torch.topk(mask_word,10,dim=1)

In [86]:
top_10

torch.return_types.topk(
values=tensor([[0.2002, 0.1785, 0.1471, 0.0475, 0.0421, 0.0320, 0.0178, 0.0154, 0.0134,
         0.0123]], grad_fn=<TopkBackward0>),
indices=tensor([[ 3994,  4940,  7717,  7364,  1996, 13107,  8666, 14005, 14135, 20368]]))

In [87]:
top_10=top_10[1][0]
print(top_10)

tensor([ 3994,  4940,  7717,  7364,  1996, 13107,  8666, 14005, 14135, 20368])


In [88]:
for token in top_10:
    # print(token)
    _word = tokenizer.decode([token])
    new_sentence = text.replace(tokenizer.mask_token,_word)
    print(new_sentence)

The opera house in Australia is in, sydney city
The opera house in Australia is in, melbourne city
The opera house in Australia is in, brisbane city
The opera house in Australia is in, adelaide city
The opera house in Australia is in, the city
The opera house in Australia is in, canberra city
The opera house in Australia is in, auckland city
The opera house in Australia is in, hobart city
The opera house in Australia is in, griffith city
The opera house in Australia is in, hume city
